In [1]:
#1. Import the necessary libraries 
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from openpyxl import load_workbook
from urllib import parse 
from datetime import datetime

import os
import io
import numpy as np
import pandas as pd
import json

#2a. Set up credentials:

# Get environmental variables:
USERNAME = os.environ.get('O365_CCR_USERNAME')
PASSWORD = os.environ.get('O365_CCR_PASSWORD')
ROOT=os.environ.get('OneDrive') #the resulting CSVs are loaded in OneDrive before they are uploaded to SharePoint


#read json config file:
with open("config.json") as config_file:
    config = json.load(config_file)
    config = config["share_point"]

#extract variables from the json file to use them in code:
url_hprs=config["site"]
selected_folder=config["inditracks_folder"] #the intermediary OneDrive folder where the resulting CSVs are stored before they are uploaded into SharePoint

file_destination=ROOT+selected_folder

#2b. creates a dataframe with the sites and lists to be iterated:
with open("General_ShP_sites.json") as sites_file:
    shp_sites = json.load(sites_file)

df_shp_sites=pd.DataFrame(shp_sites)   

current_inditracks=pd.read_csv("Current_IndiTracks.csv", index_col=0)
current_inditracks=current_inditracks.reset_index()
timestamp=datetime.now().strftime("%d/%m/%Y %H:%M:%S")


In [2]:
#3. Define function to connect to the SharePoint site:
def autenticate_in_sharepoint(shp_url, USERNAME, PASSWORD):    
    ctx_auth = AuthenticationContext(shp_url)
    if ctx_auth.acquire_token_for_user(USERNAME, PASSWORD):
        ctx = ClientContext(shp_url, ctx_auth)
        web = ctx.web
        ctx.load(web)
        ctx.execute_query()
        print ("")
        print ("")
        print ("")
        print ("")

        print("The Automat has connected to: {0}".format(web.properties['Title']))

    else:
        print ("")
        print (ctx_auth.get_last_error())
        
    return ctx

In [3]:
#4. Define function to process all tables and produce project dataframes:

def download_inditrack (ctx, project_code, relative_url):
    
    print ("")
    print("Project " + project_code + " is being processed...")
    
    try:

        #4a. Download the file:

        response = File.open_binary(ctx, relative_url)

        #save data to BytesIO stream
        bytes_file_obj = io.BytesIO()
        bytes_file_obj.write(response.content)
        bytes_file_obj.seek(0) #set file object to start


        #4b. Importing Excel spreadhseets with Pandas
        #data=pd.ExcelFile(bytes_file_obj)
        #print(data.sheet_names)

        #4c. Importing Excel´s table objects inside of the workbook with openpyxl:

        wb = load_workbook(bytes_file_obj, data_only=True)

        print ("")
        print("These are the available sheets in " + project_code + "´s workbook:")
        print(wb.sheetnames)

        ws1 = wb["M&E Plan"] #explore one specific sheet
        #print(ws1.tables.items()) # list the tables inside of the selected sheet
        ws2 = wb["Events"] #explore one specific sheet
        #print(ws2.tables.items()) # list the tables inside of the selected sheet
        ws3 = wb["Milestones"] #explore one specific sheet
        #print(ws3.tables.items()) # list the tables inside of the selected sheet

        sheets_list=[ws1,ws2,ws3]

        #4d. Creating dataframes out of each excel table:
        mapping = {}
        for sheet in sheets_list:
            for entry, data_boundary in sheet.tables.items():
                #print(entry)
                #print(data_boundary)
                #parse the data within the ref boundary
                data = sheet[data_boundary]
                #extract the data 
                #the inner list comprehension gets the values for each cell in the table
                content = [[cell.value for cell in ent] 
                           for ent in data
                      ]

                header = content[0]

                #the contents ... excluding the header
                rest = content[1:]

                #create dataframe with the column names
                #and pair table name with dataframe
                df = pd.DataFrame(rest, columns = header)
                mapping[entry] = df

        Outcomes, Activities, Indicators, Milestones=mapping.values()

        return Outcomes,Activities, Indicators, Milestones 

    except: 
        print("It has not been possible to load " + project_code + "'s workbook         #WARNING: failure on load in " + project_code)

In [4]:
#5. Define function that creates the DIM table for outcomes of type "progress":


def dim_outc_progress (Outcomes, project_code, timestamp):
    try:
        DIM_outcome_progress = Outcomes
        DIM_outcome_progress ["Indicator level"]="Outcomes"
        DIM_outcome_progress ["Project Code"]=project_code
        DIM_outcome_progress ["Version"]= timestamp
        DIM_outcome_progress ["indicator_key"]=DIM_outcome_progress ["Project Code"]+"_"+DIM_outcome_progress ["Indicator ID"]

        DIM_outcome_progress = Outcomes[Outcomes["Indicator Type"]=="Progress"][["indicator_key","Project Code","Indicator level", "Outcome ID", "Outcomes (O)",
                                            "Indicator ID","Definition of the indicator", "Overall target", "Indicator Total Weight in LogFrame)",
                                            "Specification of the indicator (if needed after reading column D)","Source of verification",
                                            "Predicted number of events within this indicator", "Indicator Type", "Version"]]

        DIM_outcome_progress.rename(columns = {'Indicator Total Weight in LogFrame)':'Indicator Total Weight in LogFrame'}, inplace=True)


        DIM_outcome_progress_indexed=DIM_outcome_progress.set_index("indicator_key")
        csv_name=project_code+"_DIM_outcome_progress.csv"
        DIM_outcome_progress_indexed.to_csv(file_destination+csv_name)
        upload_file_to_sharepoint(file_destination,csv_name)

        
        print ("")
        print ("Setting outcome dataframes for " + project_code)
        print ("")
        print ("    'DIM_outcome_progress' created...")
        return DIM_outcome_progress
    except:       
        print ("    'DIM_outcome_progress' creation failed...         #WARNING: failure on load in " + project_code)

In [5]:
#6. Define function that creates the dataframe "outc_expected_time" 
#by merging DIM Outcomes and Indicators, filtering, and implementing some adjustments:

def outc_exp_time(DIM_outcome_progress, Indicators):
    try:
        outc_expected_time=DIM_outcome_progress.merge(Indicators, on="Indicator ID")
        outc_expected_time["event_key"]=outc_expected_time ["Project Code"]+"_"+outc_expected_time ["Event ID"]


        outc_expected_time= outc_expected_time[["event_key","Project Code","Activity_ID/Outcome_ID","Indicator ID","Description","Event ID",
                                                    "Planned date of the event (what is expected)",
                                                    "Planned value of the indicator (what is expected)", "Indicator level", 
                                                    "Overall target","Indicator Total Weight in LogFrame","Indicator Type",
                                                    "Version"]]

        outc_expected_time=outc_expected_time[(outc_expected_time["Indicator level"]=="Outcomes") & (outc_expected_time["Indicator Type"]=="Progress")]

        #Creates calculated column "Expected_progress_Outc"
        outc_expected_time["Expected_progress_Outc"]=outc_expected_time["Planned value of the indicator (what is expected)"]/outc_expected_time["Overall target"]
        #Creates calculated column "Expected_Progress_Outc_Weighted"
        outc_expected_time["Expected_Progress_Outc_Weighted"]=outc_expected_time["Expected_progress_Outc"]*outc_expected_time["Indicator Total Weight in LogFrame"]
        #Format universal time into "dd/mm/yyyy"
        outc_expected_time["Planned date of the event (what is expected)"]=outc_expected_time["Planned date of the event (what is expected)"].dt.strftime('%d/%m/%Y')


        outc_expected_time_indexed=outc_expected_time.set_index("event_key")
        csv_name=project_code+"_outc_expected_time.csv"
        outc_expected_time_indexed.to_csv(file_destination+csv_name)
        upload_file_to_sharepoint(file_destination,csv_name)


        print ("    'outc_expected_time' created...")

        return outc_expected_time
    
    except:       
        print ("    'outc_expected_time' creation failed...         #WARNING: failure on load in " + project_code)
    

In [6]:
#7. Define function that creates the dataframe "outc_reported_time" 
#by merging DIM Outcomes and Indicators, filtering, and implementing some adjustments:


def outc_rep_time(DIM_outcome_progress, Indicators):
    try:
        outc_reported_time=DIM_outcome_progress.merge(Indicators, on="Indicator ID")
        outc_reported_time["event_key"]=outc_reported_time ["Project Code"]+"_"+outc_reported_time ["Event ID"]

        outc_reported_time= outc_reported_time[["event_key","Project Code","Activity_ID/Outcome_ID","Indicator ID","Description","Event ID",
                                                    "Actual date of the event (when it really happened)",
                                                    "Value of the indicator (real value after event)", "Indicator level", 
                                                    "Overall target","Indicator Total Weight in LogFrame","Indicator Type",
                                                    "Version"]]

        outc_reported_time=outc_reported_time[(outc_reported_time["Indicator level"]=="Outcomes") & (outc_reported_time["Indicator Type"]=="Progress")]

        #Drop non-reported values:
        outc_reported_time=outc_reported_time.dropna()

        
        #Create calculated column "Real Progress"
        outc_reported_time["Real Progress"]=outc_reported_time["Value of the indicator (real value after event)"]/outc_reported_time["Overall target"]

        #Format universal time into "dd/mm/yyyy"
        outc_reported_time["Actual date of the event (when it really happened)"]=outc_reported_time["Actual date of the event (when it really happened)"].dt.strftime('%d/%m/%Y')

        outc_reported_time_indexed=outc_reported_time.set_index("event_key")
        csv_name=project_code+"_outc_reported_time.csv"
        outc_reported_time_indexed.to_csv(file_destination+csv_name)
        upload_file_to_sharepoint(file_destination,csv_name)

        
        print ("    'outc_reported_time' created...")

        return outc_reported_time
    
    except:       
        print ("    'outc_reported_time' creation failed...         #WARNING: failure on load in " + project_code)

In [7]:
#8. Define function that creates the dataframe "Outcome_Progress_Top" through additions to "outc_reported_time":

def outc_time_top(DIM_outcome_progress, outc_reported_time):
    try:
        outcome_progress_top = outc_reported_time[["Indicator ID","Real Progress"]]

        outcome_progress_top=outcome_progress_top.groupby(by="Indicator ID",dropna=True).sum()

        outcome_progress_top.rename(columns = {'Real Progress':'Sum_of_Progress'}, inplace=True)

        outcome_progress_top= outcome_progress_top.merge(DIM_outcome_progress, on="Indicator ID")

        outcome_progress_top=outcome_progress_top[["indicator_key","Project Code", "Indicator ID", "Indicator Total Weight in LogFrame", "Sum_of_Progress", "Version"]]

        outcome_progress_top_indexed=outcome_progress_top.set_index("indicator_key", inplace=True)
        csv_name=project_code+"_outc_progress_top.csv"
        outcome_progress_top.to_csv(file_destination+csv_name)
        upload_file_to_sharepoint(file_destination,csv_name)
        
        
        print ("    'outcome_progress_top' created...")

        return outcome_progress_top

    except:       
        print ("    'outcome_progress_top' creation failed...         #WARNING: failure on load in " + project_code)

In [8]:
#9. Define function that creates the DIM dataframe for activity indicators of type "progress":


def dim_act_progress (Activities, project_code, timestamp):

    try:
        DIM_activities_progress = Activities

        DIM_activities_progress ["Indicator level"]="Activities"
        DIM_activities_progress ["Project Code"]=project_code
        DIM_activities_progress ["Version"]= timestamp
        DIM_activities_progress ["indicator_key"]=DIM_activities_progress["Project Code"]+"_"+DIM_activities_progress["Indicator ID"]

        #Filter "progress" rows and select columns
        DIM_activities_progress = Activities[Activities["Indicator Type"]=="Progress"][["indicator_key","Project Code","Indicator level", 
                                            "Activity ID", "Activities (Output) (A)", "Indicator ID","Definition of the indicator",
                                            "Overall target", "Indicator Total Weight in LogFrame",
                                            "Specification of the indicator (if needed after reading column D)","Source of verification", 
                                            "Predicted number of events within this indicator", "Indicator Type","Version"]]

        DIM_activities_progress_indexed=DIM_activities_progress.set_index("indicator_key")
        csv_name=project_code+"_DIM_activities_progress.csv"
        DIM_activities_progress_indexed.to_csv(file_destination+csv_name)
        upload_file_to_sharepoint(file_destination,csv_name)
       
    
        print ("")
        print ("Setting activities dataframes for " + project_code)
        print ("")
        print ("    'DIM_activities_progress' created...")

        return DIM_activities_progress
    
    except:
        print ("    'DIM_activities_progress' creation failed...         #WARNING: failure on load in " + project_code)        

In [9]:
#10. Define function that creates the dataframe "act_expected_time" 
#by merging DIM Activities and Indicators, filtering, and implementing some adjustments:

def act_exp_time(DIM_activities_progress, Indicators):
    try:
        act_expected_time=DIM_activities_progress.merge(Indicators, on="Indicator ID")
        act_expected_time["event_key"]=act_expected_time["Project Code"]+"_" +act_expected_time["Event ID"]

        act_expected_time= act_expected_time[["event_key","Project Code","Activity_ID/Outcome_ID","Indicator ID","Description","Event ID",
                                                    "Planned date of the event (what is expected)",
                                                    "Planned value of the indicator (what is expected)", "Indicator level", 
                                                    "Overall target","Indicator Total Weight in LogFrame","Indicator Type","Version"]]

        act_expected_time=act_expected_time[(act_expected_time["Indicator level"]=="Activities") & (act_expected_time["Indicator Type"]=="Progress")]

        #Creates calculated column "Expected_progress"
        act_expected_time["Expected_progress"]=act_expected_time["Planned value of the indicator (what is expected)"]/act_expected_time["Overall target"]
        #Creates calculated column "Expected_Progress_Weighted"
        act_expected_time["Expected_Progress_Weighted"]=act_expected_time["Expected_progress"]*act_expected_time["Indicator Total Weight in LogFrame"]
        #Format universal time into "dd/mm/yyyy"
        act_expected_time["Planned date of the event (what is expected)"]=act_expected_time["Planned date of the event (what is expected)"].dt.strftime('%d/%m/%Y')

        act_expected_time_indexed=act_expected_time.set_index("event_key")
        csv_name=project_code+"_act_expected_time.csv"
        act_expected_time_indexed.to_csv(file_destination+csv_name)
        upload_file_to_sharepoint(file_destination,csv_name)

        
        
        print ("    'act_expected_time' created...")


        return act_expected_time
   
    except:       
        print ("    'act_expected_time' creation failed...         #WARNING: failure on load in " + project_code)        

In [10]:
#11. Define function that creates the dataframe "act_reported_time" 
#by merging DIM Activities and Indicators, filtering, and implementing some adjustments:


def act_rep_time(DIM_activities_progress, Indicators):

    try:
        act_reported_time=DIM_activities_progress.merge(Indicators, on="Indicator ID")
        act_reported_time["event_key"]=act_reported_time["Project Code"]+"_" +act_reported_time["Event ID"]

        act_reported_time= act_reported_time[["event_key","Project Code", "Activity_ID/Outcome_ID","Indicator ID","Description","Event ID",
                                                    "Actual date of the event (when it really happened)",
                                                    "Value of the indicator (real value after event)", "Indicator level", 
                                                    "Overall target","Indicator Total Weight in LogFrame","Indicator Type", "Version"]]

        act_reported_time=act_reported_time[(act_reported_time["Indicator level"]=="Activities") & (act_reported_time["Indicator Type"]=="Progress")]

        #Drop non-reported values:
        act_reported_time=act_reported_time.dropna()

        #Create calculated column "Real_progress"
        act_reported_time["Real_progress"]=act_reported_time["Value of the indicator (real value after event)"]/act_reported_time["Overall target"]

        #Format universal time into "dd/mm/yyyy"
        act_reported_time["Actual date of the event (when it really happened)"]=act_reported_time["Actual date of the event (when it really happened)"].dt.strftime('%d/%m/%Y')
        
        act_reported_time_indexed=act_reported_time.set_index("event_key")
        csv_name=project_code+"_act_reported_time.csv"
        act_reported_time_indexed.to_csv(file_destination+csv_name)
        upload_file_to_sharepoint(file_destination,csv_name)

        
        
        print ("    'act_reported_time' created...")

        return act_reported_time
    
    except:
        print ("    'act_reported_time' creation failed...         #WARNING: failure on load in " + project_code)                

In [11]:
#12. Define a function that creates the dataframe "Activity_Progress_Top" through additions to "act_reported_time":

def act_time_top(DIM_activities_progress, act_reported_time):

    try:

        activity_progress_top = act_reported_time[["Indicator ID","Real_progress"]]

        activity_progress_top=activity_progress_top.groupby(by="Indicator ID").sum()

        activity_progress_top.rename(columns = {'Real_progress':'Sum_of_Progress'}, inplace=True)

        activity_progress_top= activity_progress_top.merge(DIM_activities_progress, on="Indicator ID")

        activity_progress_top=activity_progress_top[["indicator_key","Project Code", "Indicator ID", "Indicator Total Weight in LogFrame", "Sum_of_Progress", "Version"]]

        activity_progress_top_indexed=activity_progress_top.set_index("Indicator ID", inplace=True)
        csv_name=project_code+"_activity_progress_top.csv"
        activity_progress_top.to_csv(file_destination+csv_name)
        upload_file_to_sharepoint(file_destination,csv_name)

        
        print ("    'activity_progress_top' created...")

        return activity_progress_top

    except:       
        print ("    'activity_progress_top' creation failed...         #WARNING: failure on load in " + project_code)                

In [12]:
#13. Define a function that creates the dataframe "Milestones":
def milestones_df(Milestones,project_code, timestamp):
    try:
        milestones=Milestones
        milestones.rename(columns = {'Progress expected':'Progress expected (%)'}, inplace=True)
        milestones["Project Code"]=project_code
        milestones["Version"]=timestamp
        milestones["milestone_key"]=milestones["Project Code"]+milestones["Event ID"]
        milestones=milestones[["milestone_key","Project Code","Indicator ID","Event ID","Description of the milestone","Progress expected (%)","Version"]]

        milestones_indexed=milestones.set_index("milestone_key")
        csv_name=project_code+"_milestones.csv"
        milestones_indexed.to_csv(file_destination+csv_name)
        upload_file_to_sharepoint(file_destination,csv_name)


        print ("")
        print ("Setting milestones dataframe for " + project_code)
        print ("")
        print ("    'milestones' created...")

        return milestones

    except:       
        print ("    'milestones' creation failed...         #WARNING: failure on load")                

In [13]:
#14. Steps to upload the resulting CSV files to SharePoint:
def upload_file_to_sharepoint(source_folder,file_name):
      
    #a. Reads the file in OneDrive:
    path=source_folder+file_name
    with open(path, 'rb') as content_file:
        file_content = content_file.read()
    
    #b. Connecting to the desired folder in the tennant:
    target_url="/sites/Group-HPRS/Sdilene%20dokumenty/Data_loads-Do_not_modify/individual_inditracks"
    target_folder = ctx2.web.get_folder_by_server_relative_url(target_url)

    #c. Upload the file to SharePoint
    name = os.path.basename(path)
    target_file = target_folder.upload_file(name, file_content).execute_query()    
    
    
    

In [14]:
#15. Run program and create individual CSVs for each project:
print(current_inditracks[['Country', 'Project_code']])
ctx2=autenticate_in_sharepoint(url_hprs,USERNAME,PASSWORD) #function n.3, for uploading docs to HPRS SharePoint

for index, row in df_shp_sites.iterrows():
    shp_country = row['country']
    url = row['site']
    try:
        ctx=autenticate_in_sharepoint(url,USERNAME,PASSWORD) #function n.3, for dowloading tables from each Country SharePoint

        for index, row in current_inditracks.iterrows():
            project_code= row['Project_code']
            inditrack_country=row['Country']
            relative_url=row['IndiTrack_Relative_url']

            if shp_country ==inditrack_country:

                try:

                    Outcomes,Activities, Indicators, Milestones = download_inditrack(ctx, project_code, relative_url) #function n.4

                    #14a. Create csv for DIM_outcome_progress: 
                    DIM_outcome_progress =dim_outc_progress(Outcomes, project_code, timestamp) #function n.5

                    #14b. Create csv for outc_expected_time:
                    outc_expected_time = outc_exp_time(DIM_outcome_progress, Indicators) #function n.6

                    #14c. Create csv for outc_reported_time:
                    outc_reported_time = outc_rep_time(DIM_outcome_progress, Indicators) #function n.7

                    #14d. Create csv for outcome_progress_top:
                    outcome_progress_top = outc_time_top(DIM_outcome_progress, outc_reported_time) #function n.8

                    #14e. Create csv for DIM_activities_progress:
                    DIM_activities_progress = dim_act_progress (Activities, project_code, timestamp) #function n.9

                    #14f. Create csv for act_expected_time:
                    act_expected_time = act_exp_time(DIM_activities_progress, Indicators) #function n.10

                    #14g. Create csv for act_reported_time:
                    act_reported_time = act_rep_time(DIM_activities_progress, Indicators) #function n.11

                    #14h. Create csv for activity_progress_top:
                    activity_progress_top = act_time_top(DIM_activities_progress, act_reported_time) #function n.12

                    #14i. Create csv for milestones:
                    milestones= milestones_df(Milestones,project_code, timestamp) #function n.13

                    print ("")
                    print('Iteration for ' + project_code + ' has been sucessful')        

                except:       
                    print ("")
                    print('Iteration for ' + project_code + ' has been cancelled')

                    
    except:
        print("The Automat has failed to connect to " + shp_country + "'s SharePoint site")
        

     Country Project_code
0    Moldova         MD26
1    Moldova         MD27
2    Georgia         GE41
3    Georgia         GE43
4    Georgia         GE46
5    Georgia         GE52
6     Zambia         ZM18
7     Zambia         ZM25
8     Zambia         ZM29
9     Zambia         ZM32
10  Mongolia         MN20
11      Iraq         IR24
12     Iraq2         IR23
13  Mongolia         MN19
14   Georgia         GE53




The Golem has connected to: Group - HPRS




The Golem has connected to: Group - Georgia

Project GE41 is being processed...


C:\Users\gincas\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)



These are the available sheets in GE41´s workbook:
['M&E Plan', 'Events', 'Milestones', 'SoV - optional', 'Data dictionary']

Setting outcome dataframes for GE41

    'DIM_outcome_progress' created...
    'outc_expected_time' created...
    'outc_reported_time' created...
    'outcome_progress_top' created...

Setting activities dataframes for GE41

    'DIM_activities_progress' created...
    'act_expected_time' created...
    'act_reported_time' created...
    'activity_progress_top' created...

Setting milestones dataframe for GE41

    'milestones' created...

Iteration for GE41 has been sucessful

Project GE43 is being processed...


C:\Users\gincas\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)



These are the available sheets in GE43´s workbook:
['M&E Plan', 'Events', 'Milestones', 'SoV - optional', 'Data dictionary']

Setting outcome dataframes for GE43

    'DIM_outcome_progress' created...
    'outc_expected_time' created...
    'outc_reported_time' created...
    'outcome_progress_top' created...

Setting activities dataframes for GE43

    'DIM_activities_progress' created...
    'act_expected_time' created...
    'act_reported_time' created...
    'activity_progress_top' created...

Setting milestones dataframe for GE43

    'milestones' created...

Iteration for GE43 has been sucessful

Project GE46 is being processed...

These are the available sheets in GE46´s workbook:
['M&E Plan', 'Events', 'Milestones', 'SoV - optional', 'Data dictionary']

Setting outcome dataframes for GE46

    'DIM_outcome_progress' created...
    'outc_expected_time' created...
    'outc_reported_time' created...
    'outcome_progress_top' created...

Setting activities dataframes for GE46

 

...


These are the available sheets in IR24´s workbook:
['M&E Plan', 'Events', 'Milestones', 'SoV - optional', 'Data dictionary']

Setting outcome dataframes for IR24

    'DIM_outcome_progress' created...
    'outc_expected_time' created...
    'outc_reported_time' created...
    'outcome_progress_top' created...

Setting activities dataframes for IR24

    'DIM_activities_progress' created...
    'act_expected_time' created...
    'act_reported_time' created...
    'activity_progress_top' created...

Setting milestones dataframe for IR24

    'milestones' created...

Iteration for IR24 has been sucessful




The Golem has connected to: Group - Mongolia

Project MN20 is being processed...

These are the available sheets in MN20´s workbook:
['M&E Plan', 'Events', 'Milestones', 'SoV - optional', 'Data dictionary']

Setting outcome dataframes for MN20

    'DIM_outcome_progress' created...
    'outc_expected_time' created...
    'outc_reported_time' created...
    'outcome_progress_top' crea

C:\Users\gincas\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)



These are the available sheets in MN19´s workbook:
['M&E Plan', 'Events', 'Milestones', 'SoV - optional', 'Data dictionary']

Setting outcome dataframes for MN19

    'DIM_outcome_progress' created...
    'outc_expected_time' created...
    'outc_reported_time' created...
    'outcome_progress_top' created...

Setting activities dataframes for MN19

    'DIM_activities_progress' created...
    'act_expected_time' created...
    'act_reported_time' created...
    'activity_progress_top' created...

Setting milestones dataframe for MN19

    'milestones' created...

Iteration for MN19 has been sucessful




The Golem has connected to: Group - Moldova

Project MD26 is being processed...

These are the available sheets in MD26´s workbook:
['M&E Plan', 'Events', 'Milestones', 'SoV - optional', 'Data dictionary']

Setting outcome dataframes for MD26

    'DIM_outcome_progress' created...
    'outc_expected_time' created...
    'outc_reported_time' created...
    'outcome_progress_top' creat

C:\Users\gincas\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\gincas\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)



These are the available sheets in IR23´s workbook:
['M&E Plan', 'Events', 'Milestones', 'SoV - optional', 'Data dictionary']

Setting outcome dataframes for IR23

    'DIM_outcome_progress' created...
    'outc_expected_time' created...
    'outc_reported_time' created...
    'outcome_progress_top' created...

Setting activities dataframes for IR23

    'DIM_activities_progress' created...
    'act_expected_time' created...
    'act_reported_time' created...
    'activity_progress_top' created...

Setting milestones dataframe for IR23

    'milestones' created...

Iteration for IR23 has been sucessful
